<a href="https://colab.research.google.com/github/aeleraqi/AudioRefine/blob/main/AudioRefine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install Dependencies and Import Libraries


In [ ]:
!pip install pydub ipywidgets noisereduce
!apt-get install ffmpeg -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00
Reading package lists... Done


In [ ]:
# Import necessary libraries
from pydub import AudioSegment, effects
from pydub.silence import split_on_silence
from IPython.display import Audio, display, HTML
import io
import base64
import librosa
import librosa.display
import matplotlib.pyplot as plt
import noisereduce as nr
import soundfile as sf
import numpy as np
import warnings
from google.colab import files

###Utility Functions

In [ ]:
# Function to upload audio file
def upload_audio():
    uploaded = files.upload()
    if len(uploaded) > 0:
        file_name = list(uploaded.keys())[0]
        return file_name, uploaded
    else:
        print("The audio file was not uploaded.")
        return None, None

# Function to play audio using IPython
def play_audio(audio_data, sr):
    audio = Audio(data=audio_data, rate=sr)
    display(audio)

# Function to plot waveform
def plot_waveform(y, sr, title="Waveform"):
    plt.figure(figsize=(12, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title(title)
    plt.show()

###Audio Processing and Enhancement

In [ ]:
# Function to process and enhance audio
def process_audio(file_name, uploaded):
    # Load the uploaded audio file as a byte stream
    for filename, audio_data in uploaded.items():
        audio_data = io.BytesIO(audio_data)

    # Load audio from the byte stream
    audio = AudioSegment.from_file(audio_data)

    # Export the audio as a temporary wav file for processing with librosa
    temp_audio_path = '/content/temp_audio.wav'
    audio.export(temp_audio_path, format='wav')

    # Load the audio with librosa
    y, sr = librosa.load(temp_audio_path)

    # Estimate the noise profile (using the first 1 second as a noise sample)
    noise_sample = y[0:int(sr)]
    reduced_noise = nr.reduce_noise(y=y, sr=sr, y_noise=noise_sample)

    # Save the cleaned audio back to a file
    clean_audio_path = '/content/clean_audio.wav'
    sf.write(clean_audio_path, reduced_noise, sr)

    # Convert the cleaned audio back to an AudioSegment for further enhancement
    clean_audio_segment = AudioSegment.from_file(clean_audio_path, format="wav")

    # Apply effects
    normalized_audio = effects.normalize(clean_audio_segment)
    equalized_audio = normalized_audio.low_pass_filter(3000).high_pass_filter(100)
    compressed_audio = effects.compress_dynamic_range(equalized_audio)

    # Optional: Apply a subtle reverb effect
    delay_ms = 50
    delayed_audio = compressed_audio[:-delay_ms]
    delayed_audio = delayed_audio + AudioSegment.silent(duration=delay_ms)
    reverb = compressed_audio.overlay(delayed_audio, gain_during_overlay=-6)

    # Save the enhanced audio to a new file
    enhanced_audio_path = '/content/enhanced_audio.wav'
    reverb.export(enhanced_audio_path, format='wav')

    # Convert the enhanced audio to MP3
    enhanced_audio_segment = AudioSegment.from_file(enhanced_audio_path, format="wav")
    output_audio_path = '/content/enhanced_audio.mp3'
    enhanced_audio_segment.export(output_audio_path, format='mp3')

    return enhanced_audio_path, output_audio_path


###Display Functions

In [ ]:
# Function to display download link for enhanced audio
def display_download_link(output_audio_path):
    download_link = HTML(f'<a href="data:audio/mpeg;base64,{base64.b64encode(open(output_audio_path, "rb").read()).decode()}" download="enhanced_audio.mp3">Download enhanced audio</a>')
    display(download_link)


###Main Execution Flow

In [ ]:
def main():
    file_name, uploaded = upload_audio()
    if file_name and uploaded:
        # Temporarily suppress warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            # Load and play the original audio
            y, sr = librosa.load(file_name)
            play_audio(y, sr)
            plot_waveform(y, sr, title="Original Audio Waveform")

            # Process and enhance the audio
            enhanced_audio_path, output_audio_path = process_audio(file_name, uploaded)

            # Play and display the enhanced audio
            y_enhanced, sr_enhanced = librosa.load(enhanced_audio_path)
            play_audio(y_enhanced, sr_enhanced)
            plot_waveform(y_enhanced, sr_enhanced, title="Enhanced Audio Waveform")

            # Display download link
            display_download_link(output_audio_path)

# Run the main function
main()